In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
swap_filt = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [4]:
#ABILoading().get_abi_by_filename(abi_path)

In [5]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()
start = start - 25
print(f'latest_block {latest_block}, start {start}, end {end}')

latest_block 61234398, start 61234374, end 61234398


In [6]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=swap_filt)

In [7]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])
    event['details']['amount0In'] = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    event['details']['amount1Out'] = Conversion().convert_int256_bytes_to_int(arguments[3])  
    
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:61,234,374 tx:0xd3b48adf8fb53cf62efebecb9f07ccb158a2bbc567f46e77d6a9bbbf8261b018
Swap at block:61,234,374 tx:0xd3b48adf8fb53cf62efebecb9f07ccb158a2bbc567f46e77d6a9bbbf8261b018
Swap at block:61,234,374 tx:0xd3b48adf8fb53cf62efebecb9f07ccb158a2bbc567f46e77d6a9bbbf8261b018
Swap at block:61,234,374 tx:0xd3b48adf8fb53cf62efebecb9f07ccb158a2bbc567f46e77d6a9bbbf8261b018
Swap at block:61,234,376 tx:0x719cb90c9a0770c8cb0619939f2cbe68af3fa39cbe052cc01171a16d9f140920
Swap at block:61,234,378 tx:0xc2f97d9a9a7c4d8bd8e273806f3d3a45b8b94027a12937481dd11fd49617a8d6
Swap at block:61,234,380 tx:0x94e6abb308309ae8ccfe02d7455c79bc773e0691d8696c59a86fa1041acc575c
Swap at block:61,234,380 tx:0x4a325cee90d0011fb2f194d1a5b59caa7335e0a4a43fbae7497592ef6106712b
Swap at block:61,234,380 tx:0x4a325cee90d0011fb2f194d1a5b59caa7335e0a4a43fbae7497592ef6106712b
Swap at block:61,234,380 tx:0x4a325cee90d0011fb2f194d1a5b59caa7335e0a4a43fbae7497592ef6106712b
Swap at block:61,234,380 tx:0x4a325cee90d0011fb2f1

In [10]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,swap,sushi,0xadbf1854e5883eb8aa7baf50705338739e558e5b,0xd3b48adf8fb53cf62efebecb9f07ccb158a2bbc567f4...,61234374,1725049873,{'web3_type': <class 'web3._utils.datatypes.Sw...
1,polygon,uniswapv2pair,swap,sushi,0xadbf1854e5883eb8aa7baf50705338739e558e5b,0xd3b48adf8fb53cf62efebecb9f07ccb158a2bbc567f4...,61234374,1725049873,{'web3_type': <class 'web3._utils.datatypes.Sw...
2,polygon,uniswapv2pair,swap,sushi,0x604229c960e5cacf2aaeac8be68ac07ba9df81c3,0xd3b48adf8fb53cf62efebecb9f07ccb158a2bbc567f4...,61234374,1725049873,{'web3_type': <class 'web3._utils.datatypes.Sw...
3,polygon,uniswapv2pair,swap,sushi,0x604229c960e5cacf2aaeac8be68ac07ba9df81c3,0xd3b48adf8fb53cf62efebecb9f07ccb158a2bbc567f4...,61234374,1725049873,{'web3_type': <class 'web3._utils.datatypes.Sw...
4,polygon,uniswapv2pair,swap,sushi,0x348b9b0f279678767155fe52ecbd9e2013bc0748,0x719cb90c9a0770c8cb0619939f2cbe68af3fa39cbe05...,61234376,1725049877,{'web3_type': <class 'web3._utils.datatypes.Sw...
5,polygon,uniswapv2pair,swap,sushi,0xccb9d2100037f1253e6c1682adf7dc9944498aff,0xc2f97d9a9a7c4d8bd8e273806f3d3a45b8b94027a129...,61234378,1725049881,{'web3_type': <class 'web3._utils.datatypes.Sw...
6,polygon,uniswapv2pair,swap,sushi,0x2e99943023cf7f26eddf1ad3b9820eca5a7623dd,0x94e6abb308309ae8ccfe02d7455c79bc773e0691d869...,61234380,1725049885,{'web3_type': <class 'web3._utils.datatypes.Sw...
7,polygon,uniswapv2pair,swap,sushi,0x604229c960e5cacf2aaeac8be68ac07ba9df81c3,0x4a325cee90d0011fb2f194d1a5b59caa7335e0a4a43f...,61234380,1725049885,{'web3_type': <class 'web3._utils.datatypes.Sw...
8,polygon,uniswapv2pair,swap,sushi,0x604229c960e5cacf2aaeac8be68ac07ba9df81c3,0x4a325cee90d0011fb2f194d1a5b59caa7335e0a4a43f...,61234380,1725049885,{'web3_type': <class 'web3._utils.datatypes.Sw...
9,polygon,uniswapv2pair,swap,sushi,0xf6b87181bf250af082272e3f448ec3238746ce3d,0x4a325cee90d0011fb2f194d1a5b59caa7335e0a4a43f...,61234380,1725049885,{'web3_type': <class 'web3._utils.datatypes.Sw...


In [11]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'swap',
  'platform': 'sushi',
  'address': '0xadbf1854e5883eb8aa7baf50705338739e558e5b',
  'tx_hash': '0xd3b48adf8fb53cf62efebecb9f07ccb158a2bbc567f46e77d6a9bbbf8261b018',
  'blk_num': 61234374,
  'timestamp': 1725049873,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0xa5E0829CaCEd8fFDD4De3c43696c57F7D7A678ff',
   'token1': '0xdD7a12149c144336c6383b561ff77198574f51c4',
   'amount0In': 17000000000000000000,
   'amount1Out': 2806811961458517}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'swap',
  'platform': 'sushi',
  'address': '0xadbf1854e5883eb8aa7baf50705338739e558e5b',
  'tx_hash': '0xd3b48adf8fb53cf62efebecb9f07ccb158a2bbc567f46e77d6a9bbbf8261b018',
  'blk_num': 61234374,
  'timestamp': 1725049873,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0xa5E0829CaCEd8fFDD4De3c43696c57F7D7A678ff',
   'token1': '0xa5E0829CaCEd8fFDD4De3c43696c57F7D7A678ff',
 